In [2]:
#!/usr/bin/env python
## @package dna2aa 
#   Translate the input sequence in the specified or all frames.
#
#   Output: .fa file of translated subject sequences.
#   These sequences are naively translated as internal ORFs,
#   and are likely larger than the sequence's true ORF.
#   The output is best suited to use for hmmer-like alignments.
#
#   Alternatively, using the 'seqbreak' option, ORFs are broken
#   at stop codons.
#
#   Written by Austin Hammond, GSC 2015

import argparse
import sys
import csv

import fastatools as ft

In [2]:
def mtdna2aa(seq):
    tbl = {"ATT":"I", "ATC":"I", "ATA":"M", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
        "TTA":"L", "TTG":"L", "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V", "TTT":"F",
        "TTC":"F", "ATG":"M", "TGT":"C", "TGC":"C", "GCT":"A", "GCC":"A", "GCA":"A",
        "GCG":"A", "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G", "CCT":"P", "CCC":"P",
        "CCA":"P", "CCG":"P", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T", "TCT":"S",
        "TCC":"S", "TCA":"S", "TCG":"S", "AGT":"S", "AGC":"S", "TAT":"Y", "TAC":"Y",
        "TGG":"W", "CAA":"Q", "CAG":"Q", "AAT":"N", "AAC":"N", "CAT":"H", "CAC":"H",
        "GAA":"E", "GAG":"E", "GAT":"D", "GAC":"D", "AAA":"K", "AAG":"K", "CGT":"R",
        "CGC":"R", "CGA":"R", "CGG":"R", "AGA":"-", "AGG":"-", "TAA":"-", "TAG":"-",
        "TGA":"W"}
    j = 0 # for iteration
    aa = ''
    seql = len(seq)
    if seql % 3 != 0:
        seq = seq[0:(seql-seql%3)] #trim to multiple of 3, but should be already
    seql = len(seq) #recalculate
    for j in range(0,seql,3):
        j += 1
        if j == (seql-1):
            codon = seq[j:]
        else:
            codon = seq[j-1:j+2]
    # print codon
        aa = aa + tbl[codon]
    # force first codon to be M
    aa = "M" + aa[1:]

    return aa

In [3]:
def mtsixframe(seq):
    one = mtdna2aa(seq)
    two = mtdna2aa(seq[1:])
    three = mtdna2aa(seq[2:])
    none = mtdna2aa(ft.revcomp(seq))
    ntwo = mtdna2aa(ft.revcomp(seq)[1:])
    nthree = mtdna2aa(ft.revcomp(seq)[2:])

    return [one, two, three, none, ntwo, nthree]    

In [ ]:
# setup parser
parser = argparse.ArgumentParser(
    description='Naively translate the input DNA sequence.')
parser.add_argument(
    '--sequence', '-s', action='store', help="The fasta file of DNA sequences [STDIN]",
    default='stdin')
parser.add_argument('--frame', '-f', action='store',
    choices=['+1','+2','+3','-1','-2','-3','all'],
    help="Frame to use for sequence translation [all]", default='all')
parser.add_argument('--seqbreak','-b',action='store_true',
    help="Break translations at in-frame stop codons")
parser.add_argument('--outfile', '-o', action='store',
    help='Name to use for output fasta file [STDOUT]', default='stdout')
parser.add_argument('--mito', '-m', action='store_true', 
    help=('Sequence is mitochondrial; use vertebrate Mt code.'
        ' Does not allow stop codons to be completed by polyadenylation.'))

In [113]:
### FOR DEV ###
#frm = 'all'
outfile = 'stdout'
#outfile = 'tests/test-out.txt'
fasta = 'tests/untitled.txt'
fastadic = {}
outdic = {}
seqbreak = True
mito = False
frm = '+1'
### END DEV ###

In [ ]:
# parse args
args = parser.parse_args()
frm = args.frame
fasta = args.sequence
fastadic = {}
outfile = args.outfile
outdic = {}
mito = args.mito
seqbreak = args.seqbreak

In [ ]:
# check that the frame was passed properly
if frm == 'all':
    pass
elif "+" in frm:
    pass
elif frm == '1':
    frm = '+1'
elif frm == '2':
    frm = '+2'
elif frm == '3':
    frm = '+3'
elif "-" not in frm:
    print "ERROR: I didn't understand what reading frame you want."
    print "Please specify 'all' or a single frame including + or - and retry."
    sys.exit(1)

In [114]:
# define possible reading frames
if frm == 'all':
    frames = ['+1','+2','+3','-1','-2','-3']
else:
    frames = [frm]

In [115]:
# read in sequence input
if fasta == 'stdin':
    for rec in ft.fasta_iter(sys.stdin):
        fastadic[rec[0].split(" ")[0]] = rec[1]

else:
    with open(fasta) as fa:
        for rec in ft.fasta_iter(fa):
            sid = rec[0].split(" ")[0]
            sqn = rec[1]
            fastadic[sid] = sqn

In [116]:
if seqbreak:
    # setup separate dict to hold split translations
    transdic = {}

for sid, sqn in fastadic.items():
    if mito == True:
        if frm == 'all':
            trns = mtsixframe(sqn)
        elif frm == '+1':
            trns = mtdna2aa(sqn)
        elif frm == '+2':
            trns = mtdna2aa(sqn[1:])
        elif frm == '+3':
            trns = mtdna2aa(sqn[2:])
        elif frm == '-1':
            trns = mtdna2aa(ft.revcomp(sqn))
        elif frm == '-2':
            trns = mtdna2aa(ft.revcomp(sqn)[1:])
        else:
            trns = mtdna2aa(ft.revcomp(sqn)[2:])

    else:
        if frm == 'all':
            trns = ft.sixframe(sqn)
        elif frm == '+1':
            trns = ft.dna2aa(sqn)
        elif frm == '+2':
            trns = ft.dna2aa(sqn[1:])
        elif frm == '+3':
            trns = ft.dna2aa(sqn[2:])
        elif frm == '-1':
            trns = ft.dna2aa(ft.revcomp(sqn))
        elif frm == '-2':
            trns = ft.dna2aa(ft.revcomp(sqn)[1:])
        else:
            trns = ft.dna2aa(ft.revcomp(sqn)[2:])

    # setup output dict entry
    outdic[sid] = {}
    if seqbreak:
        # if frm == all, trns is a list; otherwise is a string #TODO make always a list
        if type(trns) == list:
            if frm == 'all':
                transzip = zip(frames, trns)
            else:
                transzip = (frm, trns)
            for rec in transzip:
                thisframe = rec[0]
                thisseq = rec[1]
                #splitsies = [x.split("-") for x in thisseq]
                splitsies = thisseq.split("-")
                # drop empty frames
                splitsies = [x for x in splitsies if x]
                outdic[sid][thisframe] = splitsies
        else:
            splitsies = trns.split("-") # split into ORFs
            multi = [x for x in splitsies if x] # drop empty ORFs
            outdic[sid][frm] = multi
    else:
        if type(trns) == list:
            outdic[sid] = {}
            if frm == 'all':
                transzip = zip(frames, trns)
            else:
                transzip = (frm, trns)
            for rec in transzip:
                thisframe = rec[0]
                thisseq = rec[1]
                outdic[sid][thisframe] = thisseq
        else:
            outdic[sid][frm] = trns

In [117]:
### FOR DEV ###
print "frm", frm
print"frames", frames
print "fastadic", fastadic
print "trns", trns
print "transzip", transzip
print "splitsies", splitsies, len(splitsies)
print "outdic", outdic
### END DEV ###

frm +1
frames ['+1']
fastadic {'seq3': 'TGGATGACACTAGGCATATAA', 'seq2': 'TGGATTACACTAGGCATATAA', 'seq1': 'ATGGATTACACTAGGCATTAA', 'seq5': 'ATGGAACATTACTAATAATAA', 'seq4': 'TGGATGACACTAGGCATATAAAAT'}
trns WMTLGI-N
transzip [('+1', 'WMTLGI-N'), ('+2', 'G-H-AYK'), ('+3', 'DDTRHIK'), ('-1', 'ILYA-CHP'), ('-2', 'FYMPSVI'), ('-3', 'FICLVSS')]
splitsies ['WMTLGI', 'N'] 2
outdic {'seq3': ['WMTLGI'], 'seq2': ['WITLGI'], 'seq1': ['MDYTRH'], 'seq5': ['MEHY'], 'seq4': ['WMTLGI', 'N']}


In [85]:
# write the translations to a file
if outfile == 'stdout':
    for nam, trns in outdic.items():
        if seqbreak:
            for frame in frames:
                seqno = 0
                for rec in trns[frame]:
                    header = "".join([">", nam, "_frame=", frame, ";ORF_num=", str(seqno + 1), "\n"])
                    print header, trns[frame][seqno]
                    seqno += 1
        else:
            for frame in frames:
                print "".join([">", nam, "_frame=", frame, "\n", trns[frame]])
else:
    with open(outfile,"w") as finalout:
        for nam, trns in outdic.items():
            if seqbreak:
                for frame in frames:
                    seqno = 0
                    for rec in trns[frame]:
                        header = "".join([">", nam, "_frame=", frame, ";ORF_num=", str(seqno + 1), "\n"])
                        print >> finalout, header, trns[frame][seqno]
                        seqno += 1
            else:
                for frame in frames:
                    print >> finalout, "".join([">", nam, "_frame=", frame, "\n", trns[frame]])

### EOF ###

>seq3_frame=+1;ORF_num=1
WMTLGI
>seq3_frame=+2;ORF_num=1
G
>seq3_frame=+2;ORF_num=2
H
>seq3_frame=+2;ORF_num=3
AY
>seq3_frame=+3;ORF_num=1
DDTRHI
>seq3_frame=-1;ORF_num=1
LYA
>seq3_frame=-1;ORF_num=2
CHP
>seq3_frame=-2;ORF_num=1
YMPSVI
>seq3_frame=-3;ORF_num=1
ICLVSS
>seq2_frame=+1;ORF_num=1
WITLGI
>seq2_frame=+2;ORF_num=1
GLH
>seq2_frame=+2;ORF_num=2
AY
>seq2_frame=+3;ORF_num=1
DYTRHI
>seq2_frame=-1;ORF_num=1
LYA
>seq2_frame=-1;ORF_num=2
CNP
>seq2_frame=-2;ORF_num=1
YMPSVI
>seq2_frame=-3;ORF_num=1
ICLV
>seq2_frame=-3;ORF_num=2
S
>seq1_frame=+1;ORF_num=1
MDYTRH
>seq1_frame=+2;ORF_num=1
WITLGI
>seq1_frame=+3;ORF_num=1
GLH
>seq1_frame=+3;ORF_num=2
AL
>seq1_frame=-1;ORF_num=1
LMPSVIH
>seq1_frame=-2;ORF_num=1
CLV
>seq1_frame=-2;ORF_num=2
S
>seq1_frame=-3;ORF_num=1
NA
>seq1_frame=-3;ORF_num=2
CNP
>seq5_frame=+1;ORF_num=1
MEHY
>seq5_frame=+2;ORF_num=1
WNITNN
>seq5_frame=+3;ORF_num=1
GTLLII
>seq5_frame=-1;ORF_num=1
LLLVMFH
>seq5_frame=-2;ORF_num=1
YY
>seq5_frame=-2;ORF_num=2
CS
>seq5_frame=-3